In [14]:
import datetime as dt
from utils import get_day_df
date = dt.datetime.now() - dt.timedelta(days=12)
s="badboll"
last_day = get_day_df(s,date.strftime("%y%m%d")) 
last_day


,Datetime,WR1_P,WR1_S1_P,WR1_sum,WR1_S1_Udc,WR2_P,WR2_S1_P,WR2_sum,WR2_S1_Udc,WR3_P,WR3_S1_P,WR3_sum,WR3_S1_Udc
0,2025-11-04 23:55:00,0,0,0,0,0,0,36361,0,0,0,37735,0
1,2025-11-04 23:50:00,0,0,0,0,0,0,36361,0,0,0,37735,0
2,2025-11-04 23:45:00,0,0,0,0,0,0,36361,0,0,0,37735,0
3,2025-11-04 23:40:00,0,0,0,0,0,0,36361,0,0,0,37735,0
4,2025-11-04 23:35:00,0,0,0,0,0,0,36361,0,0,0,37735,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2025-11-04 00:20:00,0,0,0,0,0,0,0,0,0,0,0,0
258,2025-11-04 00:15:00,0,0,0,0,0,0,0,0,0,0,0,0
259,2025-11-04 00:10:00,0,0,0,0,0,0,0,0,0,0,0,0
260,2025-11-04 00:05:00,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
import pandas as pd
import requests
import re
from io import StringIO
from datetime import datetime

def get_day_df_long(standort: str, date_str: str):
    """Lädt und parst Loggerdaten für ein gegebenes Datum & Standort, Long-Format."""

    def split_wr_column(series: pd.Series, wr: str) -> pd.DataFrame:
        """Spaltet WR-Spalte in beschriftete Teilspalten"""
        split_data = series.str.split(";", expand=True).astype(float)
        n = split_data.shape[1]
        if n < 2:
            raise ValueError(f"{wr}: Ungültige Anzahl an Werten ({n})")

        n_strings = (n - 2) // 2
        col_names = [f"{wr}_P"] \
                  + [f"{wr}_S{i+1}_P" for i in range(n_strings)] \
                  + [f"{wr}_sum"] \
                  + [f"{wr}_S{i+1}_Udc" for i in range(n_strings)]
        if (n - 2) % 2 == 1:
            col_names += [f"{wr}_T"]
        split_data.columns = col_names
        return split_data

    try:
        # URL zusammenbauen
        base_url = f"https://www.oekumenische-energiegenossenschaft.de/datenlogger/{standort}/visualisierung/"
        heute = datetime.now().strftime("%y%m%d")
        file_name = "min_day.js" if date_str == heute else f"min{date_str}.js"
        url = base_url + file_name

        # Request & Daten extrahieren
        response = requests.get(url)
        if response.status_code != 200:
            return f"Download fehlgeschlagen: {response.status_code}"

        matches = re.findall(r'="([^"]+)"', response.text)
        if not matches:
            return pd.DataFrame()

        raw_data = "\n".join(matches).replace("|", ",")
        df = pd.read_csv(StringIO(raw_data), sep=",", header=None)
        df[0] = pd.to_datetime(df[0], format="%d.%m.%y %H:%M:%S")
        df.columns = ["Datetime"] + [f"WR{i+1}" for i in range(len(df.columns) - 1)]

        # Long-Format erzeugen
        long_rows = []
        for wr in df.columns[1:]:
            split_df = split_wr_column(df[wr], wr)
            melted = split_df.melt(ignore_index=False, var_name="sensor", value_name="value")
            melted["wr"] = wr
            melted["Datetime"] = df["Datetime"]
            long_rows.append(melted[["Datetime", "wr", "sensor", "value"]])

        return pd.concat(long_rows, ignore_index=True)

    except Exception as e:
        return f"Fehler beim Parsen: {e}"
date = dt.datetime.now() #- dt.timedelta(days=12)
s="badboll"
last_day = get_day_df_long(s,date.strftime("%y%m%d")) 
last_day

,Datetime,wr,sensor,value
0,2025-11-16 16:35:00,WR1,WR1_P,0.0
1,2025-11-16 16:30:00,WR1,WR1_P,0.0
2,2025-11-16 16:25:00,WR1,WR1_P,0.0
3,2025-11-16 16:20:00,WR1,WR1_P,0.0
4,2025-11-16 16:15:00,WR1,WR1_P,0.0
...,...,...,...,...
2239,2025-11-16 00:20:00,WR3,WR3_S1_Udc,0.0
2240,2025-11-16 00:15:00,WR3,WR3_S1_Udc,0.0
2241,2025-11-16 00:10:00,WR3,WR3_S1_Udc,0.0
2242,2025-11-16 00:05:00,WR3,WR3_S1_Udc,0.0


In [19]:
import pandas as pd
import requests
import re
from io import StringIO
from datetime import datetime
import datetime as dt


def get_day_df_long(standort: str, date_str: str):
    """Lädt Loggerdaten für ein Datum & Standort, Long-Format mit Spalten: Datetime, wr, string, sensor, value."""

    def split_wr_column(series: pd.Series, wr_num: int) -> pd.DataFrame:
        """Spaltet WR-Spalte in einzelne Spalten"""
        split_data = series.str.split(";", expand=True).astype(float)
        n = split_data.shape[1]
        if n < 2:
            raise ValueError(f"WR{wr_num}: Ungültige Anzahl an Werten ({n})")

        n_strings = (n - 2) // 2
        col_names = [f"P"] \
                  + [f"S{i+1}_P" for i in range(n_strings)] \
                  + ["sum"] \
                  + [f"S{i+1}_Udc" for i in range(n_strings)]
        if (n - 2) % 2 == 1:
            col_names += ["T"]
        split_data.columns = col_names
        return split_data

    try:
        # URL zusammenbauen
        base_url = f"https://www.oekumenische-energiegenossenschaft.de/datenlogger/{standort}/visualisierung/"
        heute = datetime.now().strftime("%y%m%d")
        file_name = "min_day.js" if date_str == heute else f"min{date_str}.js"
        url = base_url + file_name

        # Request & Daten extrahieren
        response = requests.get(url)
        if response.status_code != 200:
            #print(f"Download fehlgeschlagen: {response.status_code},{url}")
            return pd.DataFrame()

        matches = re.findall(r'="([^"]+)"', response.text)
        if not matches:
            print(f"Keine Messdaten in der Datei: {url}")
            return pd.DataFrame()

        raw_data = "\n".join(matches).replace("|", ",")
        df = pd.read_csv(StringIO(raw_data), sep=",", header=None)
        df[0] = pd.to_datetime(df[0], format="%d.%m.%y %H:%M:%S")
        df.columns = ["Datetime"] + [f"WR{i+1}" for i in range(len(df.columns) - 1)]

        # Long-Format erzeugen
        long_rows = []
        for idx, wr_col in enumerate(df.columns[1:], start=1):
            split_df = split_wr_column(df[wr_col], idx)
            df_long = split_df.melt(ignore_index=False, var_name="sensor_col", value_name="value")
            # string extrahieren: Haupt-P = 1, S1_P = 2, ...
            def extract_string(s):
                if s == "P" or s == "sum" or s == "T":
                    return 1
                m = re.match(r"S(\d+)_", s)
                return int(m.group(1)) + 1 if m else 1
            df_long["string"] = df_long["sensor_col"].apply(extract_string)
            df_long["sensor"] = df_long["sensor_col"].apply(lambda x: x.split("_")[-1])
            df_long["wr"] = idx  # nur Zahl
            df_long["Datetime"] = df["Datetime"]
            long_rows.append(df_long[["Datetime", "wr", "string", "sensor", "value"]])

        return pd.concat(long_rows, ignore_index=True)

    except Exception as e:
        print(f"Fehler beim Parsen: {url}")
        return pd.DataFrame()
    
date = dt.datetime.now() #- dt.timedelta(days=12)
s="badboll"
last_day = get_day_df_long(s,date.strftime("%y%m%d")) 
last_day

,Datetime,wr,string,sensor,value
0,2025-11-16 16:35:00,1,1,P,0.0
1,2025-11-16 16:30:00,1,1,P,0.0
2,2025-11-16 16:25:00,1,1,P,0.0
3,2025-11-16 16:20:00,1,1,P,0.0
4,2025-11-16 16:15:00,1,1,P,0.0
...,...,...,...,...,...
2239,2025-11-16 00:20:00,3,2,Udc,0.0
2240,2025-11-16 00:15:00,3,2,Udc,0.0
2241,2025-11-16 00:10:00,3,2,Udc,0.0
2242,2025-11-16 00:05:00,3,2,Udc,0.0


In [37]:
from tqdm import tqdm
import datetime as dt
import pandas as pd
from tqdm import tqdm

def update_all_days():
    standorte = ["badboll","esslingen","geislingen","holzgerlingen","hospitalhof",
                 "karlsruhe","mettingen","muensingen","tuebingen","waiblingen"]
    
    end = dt.datetime.now()
    days_back = 30
    start = end - dt.timedelta(days=days_back)
    required_dates = [start + dt.timedelta(days=i) for i in range((end - start).days + 1)]

    new_data = []
    for s in tqdm(standorte):
        missing_dates = required_dates
        for date in missing_dates:
            temp = get_day_df_long(s, date.strftime("%y%m%d"))
            if isinstance(temp, pd.DataFrame) and temp.shape[0] > 0:
                # Standort-Spalte hinzufügen
                temp["standort"] = s
                new_data.append(temp)

    if new_data:
        return pd.concat(new_data, ignore_index=True).drop_duplicates()
    else:
        return pd.DataFrame()

# Beispielaufruf
res = update_all_days()
print(res.head())
res

 30%|███       | 3/10 [00:06<00:15,  2.19s/it]

Keine Messdaten in der Datei: https://www.oekumenische-energiegenossenschaft.de/datenlogger/holzgerlingen/visualisierung/min251029.js
Keine Messdaten in der Datei: https://www.oekumenische-energiegenossenschaft.de/datenlogger/holzgerlingen/visualisierung/min251030.js
Keine Messdaten in der Datei: https://www.oekumenische-energiegenossenschaft.de/datenlogger/holzgerlingen/visualisierung/min251031.js
Keine Messdaten in der Datei: https://www.oekumenische-energiegenossenschaft.de/datenlogger/holzgerlingen/visualisierung/min251101.js
Keine Messdaten in der Datei: https://www.oekumenische-energiegenossenschaft.de/datenlogger/holzgerlingen/visualisierung/min251102.js
Keine Messdaten in der Datei: https://www.oekumenische-energiegenossenschaft.de/datenlogger/holzgerlingen/visualisierung/min251103.js
Keine Messdaten in der Datei: https://www.oekumenische-energiegenossenschaft.de/datenlogger/holzgerlingen/visualisierung/min251104.js
Keine Messdaten in der Datei: https://www.oekumenische-energie

 40%|████      | 4/10 [00:08<00:12,  2.15s/it]

Keine Messdaten in der Datei: https://www.oekumenische-energiegenossenschaft.de/datenlogger/holzgerlingen/visualisierung/min251114.js
Keine Messdaten in der Datei: https://www.oekumenische-energiegenossenschaft.de/datenlogger/holzgerlingen/visualisierung/min251115.js
Keine Messdaten in der Datei: https://www.oekumenische-energiegenossenschaft.de/datenlogger/holzgerlingen/visualisierung/min_day.js


100%|██████████| 10/10 [00:23<00:00,  2.35s/it]

Keine Messdaten in der Datei: https://www.oekumenische-energiegenossenschaft.de/datenlogger/waiblingen/visualisierung/min_day.js
             Datetime  wr  string sensor  value standort
0 2025-10-17 23:55:00   1       1      P    0.0  badboll
1 2025-10-17 23:50:00   1       1      P    0.0  badboll
2 2025-10-17 23:45:00   1       1      P    0.0  badboll
3 2025-10-17 23:40:00   1       1      P    0.0  badboll
4 2025-10-17 23:35:00   1       1      P    0.0  badboll


,Datetime,wr,string,sensor,value,standort
0,2025-10-17 23:55:00,1,1,P,0.0,badboll
1,2025-10-17 23:50:00,1,1,P,0.0,badboll
2,2025-10-17 23:45:00,1,1,P,0.0,badboll
3,2025-10-17 23:40:00,1,1,P,0.0,badboll
4,2025-10-17 23:35:00,1,1,P,0.0,badboll
...,...,...,...,...,...,...
595271,2025-07-22 06:10:00,2,1,T,16.0,tuebingen
595272,2025-07-22 06:06:00,2,1,T,16.0,tuebingen
595273,2025-07-22 06:00:00,2,1,T,15.0,tuebingen
595274,2025-07-22 05:55:00,2,1,T,15.0,tuebingen


In [38]:
res.to_parquet("detaildaten.parquet",index=False)

In [51]:
df_polars = pl.scan_parquet("detaildaten.parquet")
heute = date.today()
standort="badboll"
df_filtered = (
        df_polars
        .filter(
            (pl.col("standort").str.to_lowercase() == standort.lower()) &
            (pl.col("Datetime").dt.date() == pl.lit(heute)) #&
           # (pl.col("sensor") == "P")
        )
        # .group_by("Datetime")
        # .agg(pl.col("value").sum().alias("P_sum"))
        # .sort("Datetime")
        .collect(engine="streaming")
    )

In [52]:
df_filtered.to_pandas()

,Datetime,wr,string,sensor,value,standort
0,2025-11-16 16:35:00,1,1,P,0.0,badboll
1,2025-11-16 16:30:00,1,1,P,0.0,badboll
2,2025-11-16 16:25:00,1,1,P,0.0,badboll
3,2025-11-16 16:20:00,1,1,P,0.0,badboll
4,2025-11-16 16:15:00,1,1,P,0.0,badboll
...,...,...,...,...,...,...
2239,2025-11-16 00:20:00,3,2,Udc,0.0,badboll
2240,2025-11-16 00:15:00,3,2,Udc,0.0,badboll
2241,2025-11-16 00:10:00,3,2,Udc,0.0,badboll
2242,2025-11-16 00:05:00,3,2,Udc,0.0,badboll


In [42]:
df_filtered

Datetime,P_sum
datetime[ns],f64
2025-11-16 00:00:00,0.0
2025-11-16 00:05:00,0.0
2025-11-16 00:10:00,0.0
2025-11-16 00:15:00,0.0
2025-11-16 00:20:00,0.0
…,…
2025-11-16 16:30:00,119.0
2025-11-16 16:35:00,70.0
2025-11-16 16:40:00,38.0


In [45]:
import polars as pl
from datetime import date
import numpy as np

def get_today_p_values(standort: str, file_path="detaildaten.parquet") -> np.ndarray:
    """
    Liest die Parquet-Datei im Long-Format und gibt die P-Werte des heutigen Tages
    für einen Standort als NumPy-Array zurück.
    """
    heute = date.today()

    # Datei laden als LazyFrame
    df_polars = pl.scan_parquet(file_path)

    # Filter, Gruppierung, Summierung
    df_filtered = (
        df_polars
        .filter(
            (pl.col("standort").str.to_lowercase() == standort.lower()) &
            (pl.col("Datetime").dt.date() == pl.lit(heute)) &
            (pl.col("sensor") == "P")
        )
        .group_by("Datetime")
        .agg(pl.col("value").sum().alias("P_sum"))
        .sort("Datetime")
        .collect(engine="streaming")
    )

    # NumPy-Array zurückgeben
    return np.trim_zeros(df_filtered["P_sum"].to_numpy(), 'fb')

get_today_p_values("geislingen")

array([], dtype=float64)